In [3]:
"""LCEL 체인 예제: 영화 제목 ➔ 세 개 이모지 변환

LangChain 0.1.17+ (및 langchain-core 0.1.44+) 기준 최신 방식입니다.
디프리케이티드된 `LLMChain` 없이 파이프(`|`) 연산자로 체인을 구성하고,
`RunnableWithMessageHistory` 를 올바른 경로에서 import 합니다.
"""

from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory  # ← 경로 수정

# InMemoryChatMessageHistory import 경로를 최신 버전에 맞게 수정
# langchain.memory에서 import가 안 될 경우, langchain_core.chat_history에서 import
from langchain_core.chat_history import InMemoryChatMessageHistory

# ────────────────────────────────────────────────────────────
# 1. 예시(샷) 정의
# ────────────────────────────────────────────────────────────
examples = [
    {"movie": "탑건", "emojis": "🛩️👨‍✈️🔥"},
    {"movie": "대부", "emojis": "👨‍👨‍👦🔫🍝"},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{movie}"),
    ("ai", "{emojis}"),
])

few_shot = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# ────────────────────────────────────────────────────────────
# 2. 전체 프롬프트
# ────────────────────────────────────────────────────────────
SYSTEM_MSG = (
    "당신은 영화 제목을 세 개의 이모지로만 변환해 주는 봇입니다. "
    "반드시 3개의 이모지로 답하세요. "
    "단, 사용자가 대화 내용(예: '처음에 어떤 영화를 물어봤어?')을 묻는다면 "
    "한국어 문장으로 답하십시오."
)

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_MSG),
    few_shot,                        # 예시 블록
    MessagesPlaceholder("history"), # 메모리 내용 주입
    ("human", "{movie}"),          # 최신 사용자 입력
])

# ────────────────────────────────────────────────────────────
# 3. LLM + 체인 (LCEL)
# ────────────────────────────────────────────────────────────
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
chain = prompt | llm | StrOutputParser()

# ────────────────────────────────────────────────────────────
# 4. 메모리 래퍼: RunnableWithMessageHistory
# ────────────────────────────────────────────────────────────

def memory_factory():
    """세션마다 새로운 인메모리 히스토리 생성."""
    return InMemoryChatMessageHistory()

# get_session_history 인자를 추가해야 함
def get_session_history(session_id: str):
    # 세션별로 메모리 객체를 생성 (간단 예시: 매번 새로 생성)
    return InMemoryChatMessageHistory()

chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history,  # 필수 인자 추가
    input_messages_key="movie",      # 사용자 입력 key
    history_messages_key="history",  # MessagesPlaceholder key
)

# ────────────────────────────────────────────────────────────
# 5. DEMO 실행 예시
# ────────────────────────────────────────────────────────────
if __name__ == "__main__":
    session_cfg = {"configurable": {"session_id": "demo"}}

    print("인셉션 ➜", chain_with_memory.invoke({"movie": "인셉션"}, config=session_cfg))
    print("포레스트 검프 ➜", chain_with_memory.invoke({"movie": "포레스트 검프"}, config=session_cfg))
    print("처음 질문했던 영화 ➜", chain_with_memory.invoke({"movie": "처음에 내가 어떤 영화를 물어봤는지 알려줘"}, config=session_cfg))


인셉션 ➜ 💤🌌🌀
포레스트 검프 ➜ 🏃‍♂️🍫🌳
처음 질문했던 영화 ➜ 당신은 "탑건"이라는 영화를 물어보셨습니다.
